# AI Workshop 2024
EikLab

## PART 1 - Detection

Installer ultralytics

In [ ]:
%pip install ultralytics

In [ ]:
%pip install opencv-python

In [ ]:
%pip install pillow

In [ ]:
%pip install cvzone

Importer YOLO

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8m.pt")

In [ ]:
results = model.predict("dogs.jpg")

In [ ]:
result = results[0]
print(len(result.boxes))

In [ ]:
for box in result.boxes:
    label = box.cls[0].item()
    cords = [round(x) for x in box.xyxy[0].tolist()]
    prob = box.conf[0].item()
    print("Object type:",label)
    print("Cords:",cords)
    print("Probability:",prob)
    print("---")

In [ ]:
from PIL import Image
Image.fromarray(result.plot()[:,:,::-1])

Nå kan dere prøve å kjøre YOLO på et bilde som dere har lastet opp.

## Part 2 - Segmentation

In [ ]:
from ultralytics import YOLO
import random
import cv2
import numpy as np

In [ ]:
model = YOLO("yolov8m-seg.pt")

In [ ]:
img = cv2.imread("dogs.jpg")


# if you want all classes
yolo_classes = list(model.names.values())
classes_ids = [yolo_classes.index(clas) for clas in yolo_classes]

conf = 0.5

results = model.predict(img, conf=conf)
colors = [random.choices(range(256), k=3) for _ in classes_ids]
print(results)
for result in results:
    for mask, box in zip(result.masks.xy, result.boxes):
        points = np.int32([mask])
        # cv2.polylines(img, points, True, (255, 0, 0), 1)
        color_number = classes_ids.index(int(box.cls[0]))
        cv2.fillPoly(img, points, colors[color_number])

In [ ]:
cv2.imshow("Image", img)
cv2.waitKey(0)

## Part 3 - Deteksjon av live footage

In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math

# For Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1080)
cap.set(4, 720)

# Using Videos
#cap = cv2.VideoCapture('car-videos.mp4')

model = YOLO("yolov8n.pt")

className = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck",
             "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
             "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
             "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
             "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
             "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
             "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
             "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa",
             "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse",
             "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
             "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier",
             "toothbrush"
             ]

while True:
    success, img = cap.read()
    result = model(img, stream=True)
    for r in result:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 3)

            # cvZone
            w, h = x2-x1, y2-y1
            bbox = x1, y1, w, h
            cvzone.cornerRect(img, bbox)
            # Confidence
            conf = math.ceil((box.conf[0]*100))/100
            # Class Name
            cls = int(box.cls[0])

            #Display Confidence & Class
            cvzone.putTextRect(img, f'{className[cls]} {conf}', (max(0, x1+15), max(35, y1-15)), scale= 2, thickness= 2)



    cv2.imshow("Image", img)
    cv2.waitKey(1)



## Oppgave:

Hvis det blir tid, kan dere prøve å kjøre YOLO på live footage fra kameraet på PCen.